## Drug-Kinase Interaction Prediction with CNN

## Problem:

* This is a Multi-Label Classification problem where multiple labels may be assigned to each instance.

https://nickcdryan.com/2017/01/23/multi-label-classification-a-guided-tour/

https://stats.stackexchange.com/questions/12702/what-are-the-measure-for-accuracy-of-multilabel-data?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

## Network Architecture


* INPUT->[CONV -> RELU -> CONV ->RELU ->POOL] ->FC -> RELU -> FC ->SIGMOID ->OUTPUT

## Cost Function

There are two ways to penalize the instances:
* if you do not want to miss any label in an image then if the classification gets all right but one,you should consider the whole things wrong,
* you can also that the label missed or misclassified is an error

We use the second method:

`sigmoid_cross_entropy_with_logits` is a TensorFlow function that penalizes each output node independently. It uses binary loss and model the output of the netowrk as an independed Bernouli distribution per label.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

## Load data

In [2]:
drug_fingerprints_fh = 'sample/sample_fingerprints.csv'
drug_targets_fh      = 'sample/sample_targets.csv'
drug_weights_fh      = 'sample/sample_weights.csv'

### Dimensions of data 

In [5]:
sample_size       = 10000
fingerprint_size  = 1024
fingerprint_width = 32
targets_num       = 420
weights_num       = 420
num_channels      = 1

### Function helper that populates data structures with the actual data

In [6]:
import re
def populate_data(file_handle,data_matrix, data_size):
    with open(file_handle) as fh:
        j=0
        content = fh.readlines()
        content = [x.strip() for x in content]
        for line in content:
            result = re.split(r'[,\t]\s*',line)
            for i in range(1,data_size+1):
                data_matrix[j][i-1] = np.float32(result[i])
            j = j+1
    print(j)
    fh.close()

### Data structures for loaded data

In [7]:
drug_fingerprints = []
drug_targets      = []
drug_weights      = []


for i in range(sample_size):
    fingerprint_holder = [0]* fingerprint_size
    drug_fingerprints.append(fingerprint_holder)
    
for i in range(sample_size):
    target_holder = [0]* targets_num
    drug_targets.append(target_holder)

for i in range(sample_size):
    weight_holder = [0]* weights_num
    drug_weights.append(weight_holder)

In [8]:
populate_data(drug_weights_fh, drug_weights, weights_num)
populate_data(drug_targets_fh, drug_targets, targets_num)
populate_data(drug_fingerprints_fh, drug_fingerprints, fingerprint_size)

10000
10000
10000


In [9]:
drug_fingerprints = np.array(drug_fingerprints)
drug_targets      = np.array(drug_targets)
drug_weights      = np.array(drug_weights)

# TensorFlow

## Placeholders
Placeholder for the flat 'array' with **fingerprint** of each compound. `None` means that this tensor can hold arbitrary number of arrays with the fingerprints.

In [11]:
x = tf.placeholder(tf.float32, [None, fingerprint_size],name = "Input_Flat_Kinase_1024")

X is first define as and vecor of size `fingerprint_size` and its then redefine and reshape input as a 2D matrix (image)

In [12]:
drug_image = tf.reshape(x, [-1, fingerprint_width, fingerprint_width, num_channels], name="Kinase_image_32x32")

Placeholder for the true labels (true targets) for each compound. (Here we have 420 targets - kinases).

In [11]:
y_true = tf.placeholder(tf.float32, [None, targets_num])

Placeholder for weights. The weights will be used to calculate cross-entropy cost function.

In [12]:
cross_entropy_weights = tf.placeholder(tf.float32, [None, weights_num])

## Variables to Optimize

In [13]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [14]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

## Network Architecture

* INPUT->[CONV -> RELU -> CONV ->RELU ->POOL] ->FC ->RELU->FC->SIGMOID->FC

## Helper Functions to Create the Network's Layers

### NEW CONVOLUTION LAYER

In [15]:
def new_conv_layer_with_RELU(input, num_input_channels, filter_size, num_filters):        
    
    #number and shape of the filters
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    
    #create the filters
    weights = new_weights(shape=shape)

    #create bias for each depth slice of the filter volumne
    biases = new_biases(length=num_filters)

    #convolute stride =[1,1,1,1] move by pixel, padding = "SAME" -> add zero padding to keep the dimensions
    layer = tf.nn.conv2d(input   = input,filter  = weights, strides = [1, 1, 1, 1], padding = 'SAME')
    
    # Add the biases 
    layer += biases
    
    #  non-linearity (ReLU).
    layer = tf.nn.relu(layer)
                   
    return layer

### NEW POOLING LAYER

In [16]:
def new_pooling_layer(input, stride):
    
    layer = tf.nn.max_pool(value=input,ksize=[1, 2, 2, 1],strides=[1, stride, stride, 1],padding='SAME')
    
    return layer

### Helper Function for Flattening a Layer

In [17]:
def flatten_layer(layer):
        
    # get the shape of the input layer in a format [num_images, img_height, img_width, num_channels]
    layer_shape = layer.get_shape()

    # number of features is  img_height * img_width * num_channels
    num_features = int(layer_shape[1] * layer_shape[2] * layer_shape[3])
    
    # reshape the layer to [num_images, num_features].
    # -1  means the size in that dimension is calculated so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # return both the flattened layer and the number of features.
    return layer_flat, num_features

### NEW FULLY-CONNECTED LAYER

In [18]:
def new_fc_layer(input, num_inputs,num_outputs,use_relu, use_sigmoid): 

    # new weights and biases for the layer
    weights = new_weights(shape = [num_inputs, num_outputs])
    biases = new_biases(length = num_outputs)

    # calculate the layer as the matrix multiplication of the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    if use_relu:
        layer = tf.nn.relu(layer)
       
    if use_sigmoid:
        layer = tf.nn.sigmoid(layer)

    return layer

# Design Computational Graph for CNN

### HYPER-PARAMETERS OF THE NETWORK

INPUT->[CONV -> RELU -> CONV ->RELU ->POOL] ->FC -> RELU -> FC ->SIGMOID ->OUTPUT

* **CONV 1**

In [19]:
filter_size1 = 2
num_filters1 = 16

* **CONV 2**

In [20]:
filter_size2 = 3
num_filters2 = 32

* **FC 1 & FC 2**

In [21]:
fc_size1 = 2576
fc_size2 = 420

DATA DIMENSION REMINDER:
* sample_size       = 10000
* fingerprint_size  = 1024
* fingerprint_width = 32
* targets_num       = 420
* weights_num       = 420
* num_channels      = 1

## DEFINE THE LAYERS

### Convolutional Layer #1 with RELU

In [22]:
conv_layer1 = new_conv_layer_with_RELU(input=drug_image,
                                       num_input_channels = num_channels,
                                       filter_size = filter_size1,
                                       num_filters = num_filters1 )  

In [23]:
conv_layer1

<tf.Tensor 'Relu:0' shape=(?, 32, 32, 16) dtype=float32>

### Convolutional Layer #2 with RELU

In [24]:
conv_layer2 = new_conv_layer_with_RELU(input = conv_layer1,
                                       num_input_channels = num_filters1 ,
                                       filter_size = filter_size2,
                                       num_filters = num_filters2)  

In [25]:
conv_layer2

<tf.Tensor 'Relu_1:0' shape=(?, 32, 32, 32) dtype=float32>

### Pooling Layer

In [26]:
pooling_layer = new_pooling_layer(input= conv_layer2, stride = 2)

In [27]:
pooling_layer 

<tf.Tensor 'MaxPool:0' shape=(?, 16, 16, 32) dtype=float32>

### Prepare input to FC layer aka flattering the layer

In [28]:
layer_flat, features_num = flatten_layer(pooling_layer)

In [29]:
layer_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 8192) dtype=float32>

### Fully- Connected Layer 1

In [30]:
fc_layer1 = new_fc_layer(input = layer_flat,
                         num_inputs = features_num,
                         num_outputs = fc_size1,
                         use_relu = True,
                         use_sigmoid = False)

In [31]:
fc_layer1

<tf.Tensor 'Relu_2:0' shape=(?, 2576) dtype=float32>

### Fully- Connected Layer 2

In [32]:
fc_layer2 = new_fc_layer(input = fc_layer1,
                         num_inputs = fc_size1,
                         num_outputs = fc_size2,
                         use_relu = False,
                         use_sigmoid = True)

In [33]:
fc_layer2

<tf.Tensor 'Sigmoid:0' shape=(?, 420) dtype=float32>

### OUTPUT -> Predicted Classes 

In [34]:
output = tf.round(fc_layer2)

# Cost Function to Optimize

Because we want to penalize each output node independently, we pick a binary loss and model the ouput of the network as an independent Bernouli Distribution per label.

In [35]:
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits = fc_layer2,
                                                        labels = y_true)

### Multiply logistic loss with weights (ELEMENT-WISE) 

In [36]:
# sum of cost for all labels with weight 1
cost_sum = tf.reduce_sum(tf.multiply(cross_entropy_weights,cross_entropy))

# number of labels with weight 1
num_nonzero_weights = tf.count_nonzero(input_tensor=cross_entropy_weights,dtype = tf.float32)

# average cost
cost = tf.divide(cost_sum, num_nonzero_weights)

### Optimization Method

In [37]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [38]:
accuracy, accuracy_ops =tf.metrics.accuracy(labels=y_true,predictions=output)

In [39]:
# Local variables need to show updated accuracy on each iteration 
stream_vars = [i for i in tf.local_variables()]

# Create TensorFlow session

In [40]:
session = tf.Session()
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
session.run(init)

In [41]:
train_batch_size = 50

In [42]:
def fetch_batch(batch_size):
    chosen = np.random.randint(len(drug_fingerprints), size = batch_size)
    X_batch = drug_fingerprints[chosen, :]
    y_batch = drug_targets[chosen, :]
    cross_entropy_weights = drug_weights[chosen,:]
    return X_batch,y_batch,cross_entropy_weights

In [43]:
# counter for total number of iterations
total_iterations = 0

def optimize(num_iterations):
    
    # update the global variable rather than a local copy.
    global total_iterations

    # start-time 
    start_time = time.time()

    for i in range(total_iterations, total_iterations + num_iterations):

        # batch of training examples
        x_batch, y_true_batch, weights_batch = fetch_batch(train_batch_size)

        # put the batch into a dict with the proper names for placeholder variables
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch,
                          cross_entropy_weights: weights_batch}

        # run the optimizer with the btch training data
        session.run(optimizer, feed_dict=feed_dict_train)

        # print update every 10 iterations
        if (i+1) % 10 == 0:
            
            # calculate the accuracy on the training-set.
            acc_ops = session.run(accuracy_ops, feed_dict=feed_dict_train)
            
            # print update
            print('[Total correct, Total count]:',session.run(stream_vars)) 
            print("Optimization Iteration: {}, Training Accuracy: {} \n".format(i+1,acc_ops))                        

    # update the total number of iterations
    total_iterations += num_iterations

    # end time
    end_time = time.time()

    # difference between start and end-times.
    time_dif = end_time - start_time

    #time-usage
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [44]:
optimize(num_iterations=71)

[Total correct, Total count]: [17649.0, 21000.0]
Optimization Iteration: 10, Training Accuracy: 0.8404285907745361 

[Total correct, Total count]: [37476.0, 42000.0]
Optimization Iteration: 20, Training Accuracy: 0.8922857046127319 

[Total correct, Total count]: [57884.0, 63000.0]
Optimization Iteration: 30, Training Accuracy: 0.9187936782836914 

[Total correct, Total count]: [78697.0, 84000.0]
Optimization Iteration: 40, Training Accuracy: 0.9368690252304077 

[Total correct, Total count]: [99555.0, 105000.0]
Optimization Iteration: 50, Training Accuracy: 0.9481428861618042 

[Total correct, Total count]: [120423.0, 126000.0]
Optimization Iteration: 60, Training Accuracy: 0.9557380676269531 

[Total correct, Total count]: [141284.0, 147000.0]
Optimization Iteration: 70, Training Accuracy: 0.9611156582832336 

Time usage: 0:00:08


In [45]:
writer = tf.summary.FileWriter("./logs/CNN_1", session.graph)

In [48]:
! tensorboard --logdir=logs

TensorBoard 1.12.1 at http://malina-desktop:6006 (Press CTRL+C to quit)
^C
